In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')

Connecting celiib@10.28.0.34:3306


In [9]:
#(schema.jobs & "table_name='__coarse_label_orphan'")#.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [7]:
@schema
class CoarseLabelOrphan(dj.Computed):
    definition = """
    # Vertex labels for pinky.ProofreadLabelOrphan did not correctly match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.ProofreadLabelOrphan
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.ProofreadLabelOrphan
    """
    
    key_source = pinky.ProofreadLabelOrphan & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.ProofreadLabelOrphan & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.Decimation35OrphanStitched & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [8]:
start = time.time()
CoarseLabelOrphan.populate(reserve_jobs=True)
print(time.time() - start)

0.03643441200256348


In [10]:
pinky.CoarseLabelOrphan()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices Corrected vertex labels,triangles Same triangle labels as pinky.ProofreadLabelOrphan
3,648518346341352891,0.35,computer_Auto,celiib,2019-06-23 10:05:42,=BLOB=,=BLOB=
3,648518346341353058,0.35,computer_Auto,celiib,2019-06-23 12:08:34,=BLOB=,=BLOB=
3,648518346341353574,0.35,computer_Auto,celiib,2019-06-23 13:05:53,=BLOB=,=BLOB=
3,648518346341353607,0.35,computer_Auto,celiib,2019-06-23 13:08:39,=BLOB=,=BLOB=
3,648518346341353788,0.35,computer_Auto,celiib,2019-06-23 13:09:40,=BLOB=,=BLOB=
3,648518346341354313,0.35,computer_Auto,celiib,2019-06-23 16:04:08,=BLOB=,=BLOB=
3,648518346341354496,0.35,computer_Auto,celiib,2019-06-23 16:05:29,=BLOB=,=BLOB=
3,648518346341355048,0.35,computer_Auto,celiib,2019-06-23 16:05:59,=BLOB=,=BLOB=
3,648518346341355198,0.35,computer_Auto,celiib,2019-06-23 16:06:47,=BLOB=,=BLOB=
3,648518346341355736,0.35,computer_Auto,celiib,2019-06-23 16:07:10,=BLOB=,=BLOB=


In [ ]:
#(schema.jobs & "table_name='__coarse_label_orphan'").delete()

In [ ]:
pinky.CoarseLabelOrphan()